In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
df=pd.read_csv('train_df.csv')
df.columns

Index(['УИН', 'количество этажей всего', 'высота объекта',
       'Этажность объекта подземная', 'Этажность объекта наземная',
       'Количество квартир', 'Общая площадь объекта',
       'Площадь жилых помещений объекта', 'lon', 'lat', 'pricemetr',
       'metroprox', 'rwprox', 'medprox', 'kidprox', 'schoolprox', 'cemetprox',
       'industrprox', 'liveratio', 'highratio'],
      dtype='object')

In [11]:
df.drop(df[df['pricemetr']<.1].index, inplace=True)
df.drop(df[df['pricemetr']>1.5].index, inplace=True)
Xy=df.drop(['УИН',   'высота объекта', 'количество этажей всего','Количество квартир','Этажность объекта подземная','Площадь жилых помещений объекта',
             #'liveratio', 'highratio',
             'industrprox','medprox', 'kidprox', 'schoolprox', 'cemetprox', 'rwprox',
           ], axis=1).dropna() 
Xy.rename(columns={'Этажность объекта наземная':'planchers_sur',
                   'Общая площадь объекта': 'superficie_tot'}, inplace=True)

In [12]:
X=Xy.drop(['pricemetr'], axis=1)
y=Xy.pricemetr
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((328, 7), (328,), (162, 7), (162,))

In [13]:
#model=KNeighborsRegressor(n_neighbors=2)
model=GradientBoostingRegressor(n_estimators=150, learning_rate=.05)
#model=RandomForestRegressor(n_estimators=100, min_samples_split=2, min_samples_leaf=4)

model.fit(X_train, y_train)
print('\tR^2_test:%.2f; R^2_train:%.2f'%(model.score(X_test,y_test), model.score(X_train,y_train)))

y_pred=model.predict(X_test)
y_pr_tr=model.predict(X_train)
err1, err2 = mean_absolute_percentage_error(y_test, y_pred), mean_absolute_percentage_error(y_train, y_pr_tr)
print('\tMAPE_test:%.1f%%; MAPE_train:%.1f%%'%(err1*100,err2*100))

	R^2_test:0.70; R^2_train:0.91
	MAPE_test:17.9%; MAPE_train:10.6%


In [14]:
with open('GBoost.pkl', 'wb') as f:
    pickle.dump(model, f)